In [33]:
import json, os
from datasets import load_dataset
from tqdm import tqdm

In [2]:
METADATA_FOLDER = "/scratch/lamdo/eru_kg_retrieval_eval"
PRECOMPUTE_KEYPHRASE_FOLDER = "/scratch/lamdo/precompute_keyphrase_extraction/"

In [27]:
ds = load_dataset("BeIR/trec-covid", "corpus")

ids = []
docs = []
for line in ds["corpus"]:
    doc_id = line.get("_id")
    title = line.get("title")
    _text = line.get("text")

    text = f"{title.lower()}. {_text.lower()}"
    ids.append(doc_id)
    docs.append(text)

In [28]:
id2doc = {id: doc for id, doc in zip(ids, docs)}

In [17]:
with open(os.path.join(METADATA_FOLDER, "trec_covid_keyphrase_expansion_retrieval_based_ukg_custom_trained_combined_references_nounphrase_v6-1_position_penalty+length_penalty.json")) as f:
    eru_kg = json.load(f)

with open(os.path.join(METADATA_FOLDER, "trec_covid_copyrnn-1.json")) as f:
    copyrnn = json.load(f)


with open(os.path.join(PRECOMPUTE_KEYPHRASE_FOLDER, "trec_covid--retrieval_based_ukg_custom_trained_combined_references_nounphrase_v6-1_position_penalty+length_penalty.json") )as f:
    temp = json.load(f)
    eru_kg_keyphrase = {_id: item for _id, item in zip(ids, temp)}

with open(os.path.join(PRECOMPUTE_KEYPHRASE_FOLDER, "trec_covid--copyrnn-1.json") )as f:
    temp = json.load(f)
    copyrnn_keyphrase = {_id: item for _id, item in zip(ids, temp)}

del temp

In [18]:
# get samples where copy rnn does better than eru kg
queries = list(eru_kg["scores"].keys())
queries_need_inspection = []
for query in queries:
    if copyrnn["scores"][query]["recall_1000"] > eru_kg["scores"][query]["recall_1000"]:
        print(query)
        print(eru_kg["scores"][query])
        print(copyrnn["scores"][query])
        print("\n\n\n")

        queries_need_inspection.append(query)

what causes death from covid-19?
{'P_10': 0.3, 'P_100': 0.09, 'P_1000': 0.105, 'recall_10': 0.005671077504725898, 'recall_100': 0.017013232514177693, 'recall_1000': 0.19848771266540643, 'ndcg_cut_10': 0.22784037450527975, 'ndcg_cut_100': 0.08735676502748967, 'ndcg_cut_1000': 0.1868534172922271, 'map_cut_10': 0.0015122873345935729, 'map_cut_100': 0.0034839426692600014, 'map_cut_1000': 0.026216695383003787}
{'P_10': 0.3, 'P_100': 0.1, 'P_1000': 0.12, 'recall_10': 0.005671077504725898, 'recall_100': 0.01890359168241966, 'recall_1000': 0.22684310018903592, 'ndcg_cut_10': 0.23758350840568812, 'ndcg_cut_100': 0.09711981008693348, 'ndcg_cut_1000': 0.21125880974999856, 'map_cut_10': 0.0017215770996489332, 'map_cut_100': 0.004013602904344392, 'map_cut_1000': 0.03414164287645385}




what drugs have been active against sars-cov or sars-cov-2 in animal studies?
{'P_10': 1.0, 'P_100': 0.44, 'P_1000': 0.228, 'recall_10': 0.016286644951140065, 'recall_100': 0.07166123778501629, 'recall_1000': 0.3713

In [19]:
current_check_query = queries_need_inspection[0]
current_check_query

'what causes death from covid-19?'

In [20]:
predictions = set(eru_kg["results"][current_check_query].keys())
groundtruths = set(eru_kg["qrels"][current_check_query].keys())

overlap_doc_ids_eru_kg = list(predictions.intersection(groundtruths))

In [21]:
predictions = set(copyrnn["results"][current_check_query].keys())
groundtruths = set(copyrnn["qrels"][current_check_query].keys())

overlap_doc_ids_copyrnn = list(predictions.intersection(groundtruths))

In [22]:
doc_ids_to_check = list(set(overlap_doc_ids_copyrnn) - set(overlap_doc_ids_eru_kg))

In [23]:
j = 0
eru_kg_keyphrase[doc_ids_to_check[j]], copyrnn_keyphrase[doc_ids_to_check[j]]

({'present_keyphrases': [],
  'absent_keyphrases': [],
  'automatically_extracted_keyphrases': {'present_keyphrases': ['patients',
    'dad',
    'covid-19',
    'thrombosis',
    'viral rna',
    'pulmonary arterial thrombosis',
    'liver',
    'coronavirus disease 2019',
    'fatal outcome',
    'central vein thrombosis',
    'sample',
    'clinicopathologic case series',
    'single pathology department',
    'cell proliferation',
    'autopsy',
    'liver cell necrosis',
    'bronchopneumonia',
    'sars-cov-2',
    'real-time polymerase chain reaction assay',
    'suspected pulmonary arterial thrombosis',
    'renal proximal tubular injury',
    'chronic hepatic congestion',
    'random',
    'laboratory',
    'adrenocortical hyperplasia',
    'venous thromboembolism',
    'fatal outcomes',
    'prospective autopsy study',
    'hepatic steatosis',
    'systematic macroscopic',
    'portal fibrosis',
    'viral analysis',
    'focal pancreatitis',
    'design',
    'measurements',

In [26]:
list(eru_kg_keyphrase.items())[0]

('ug7v899j',
 {'present_keyphrases': [],
  'absent_keyphrases': [],
  'automatically_extracted_keyphrases': {'present_keyphrases': ['pneumonia',
    'infection',
    'patients',
    'clinical features',
    'm pneumoniae infection',
    'm pneumoniae pneumonia',
    'culture-proven mycoplasma pneumoniae infections',
    'cough',
    'saudi arabia',
    'infections',
    'retrospective chart review',
    'malaise',
    'bronchiolitis',
    'fever',
    'positive m. pneumoniae cultures',
    'fall',
    'most patients',
    'immunocompromised patients',
    'abdulaziz university hospital',
    'bronchial breathing',
    'preschool children',
    'mortality rate',
    'pre-school children',
    'non-immunocompromised patients',
    'most infections',
    'comorbidities',
    'common symptoms',
    'crepitations',
    'upper respiratory tract infections',
    'underlying comorbidities',
    'respiratory specimens',
    'other comorbidities',
    'microbiology records',
    'epidemiology',


In [42]:
def get_absent_token_ratio(keyphrases):
    ratios = []
    for doc_id in tqdm(keyphrases):
        temp = keyphrases[doc_id].get("automatically_extracted_keyphrases")
        absent_keyphrases = temp.get("absent_keyphrases")[:10]

        doc = id2doc[doc_id]

        absent_keyphrases_containing_present_words = []
        for kw in absent_keyphrases:
            kw_tokens = kw.split(" ")
            if all([kw_token in doc for kw_token in kw_tokens]):
                absent_keyphrases_containing_present_words.append(kw)

        if not absent_keyphrases:
            continue
        else:
            r = 1 - len(absent_keyphrases_containing_present_words) / len(absent_keyphrases)
            ratios.append(r)

    return sum(ratios) / len(ratios)

In [43]:
get_absent_token_ratio(eru_kg_keyphrase), get_absent_token_ratio(copyrnn_keyphrase)

  0%|          | 0/171332 [00:00<?, ?it/s]

100%|██████████| 171332/171332 [00:03<00:00, 50082.20it/s]


(0.7767556539324123, 0.4168066297717356)